### Tarea 5. Análisis de resultados 

1. Parametrización de algoritmos:
En esta tarea vamos a comparar diferentes algoritmos para resolver problemas de optimización
continua. Implementa los cambios necesarios para poder realizar las siguientes comparaciones:

    * Parámetros del Recocido simulado: Comparar al menos dos esquemas de enfriamiento.
    * Parámetros del Algoritmo genético: Comparar los siguientes esquemas de reemplazo:
        * Generacional
        * Generacional con elitismo
        * Reemplazo de los peores
    * Mejor parametrización: Recocido simulado vs algoritmo genético


In [165]:
import numpy as np
import math
import random

#Implementamos todas las funciones a utilizar

def sphere(x):
    if x.ndim == 2 : 
        y = np.sum(x**2,axis=1)
    else: 
        y = np.sum(x**2)
    return y

def ackley(x):
    x = np.array(x)
    if x.ndim == 2 : 
        dim = x.shape[1]
        t1 = np.sum(x**2, axis=1)
        t2 = np.sum(np.cos(2 * np.pi * x), axis=1)
        y = -20 * np.exp(-0.2 * np.sqrt(t1 / dim)) - np.exp(t2 / dim) + 20 + np.exp(1)
    else: 
        t1 = -20*np.exp(-0.2*np.sqrt((1/len(x))*sum(x**2)))
        t2 = np.exp((1/len(x))*sum(np.cos(2*np.pi*x)))
        y = 20+t1-t2+np.exp(1)
    return y

def griewank(x):
    x = np.array(x)
    if x.ndim == 2: 
        i = np.array(range(1,x.shape[1]+1))
        t1 = np.sum(x**2, axis=1)/4000
        t2 = np.prod(np.cos(x/np.sqrt(i)))
    else: 
        i = np.array(range(1,len(x)+1))
        t1 = np.sum(x**2)/4000
        t2 = np.prod(np.cos(x/np.sqrt(i)))

    return 1 + t1 - t2

def rastrigin(x):
    x = np.array(x)
    if  x.ndim == 2: 
        t1 = 10*(x.shape[1])
        t2 = np.sum(x**2-10*np.cos(2*np.pi*x),axis=1)
    else: 
        i = np.array(range(1,len(x)+1))
        t1 = sum(x**2)/4000
        t2 = np.prod(np.cos(x/np.sqrt(i)))
    
    return t1 + t2

#La función aplica para parametros mayores o iguales a dos dimensiones
def rosenbrock(x):
    x = np.array(x)
    if  x.ndim == 2:
        x1 = x[:,1:]
        x0 = np.delete(x,0,axis = 1)
        t1 = 100*(x1-x0**2)**2
        t2 = (1-x0)**2 
        y = np.sum(t1 + t2,axis= 1)
    else: 
        x1 = x[1:]
        x0 = np.delete(x,-1)
        t1 = 100*(x1-x0**2)**2
        t2 = (1-x0)**2
        y = np.sum(t1 + t2)
    
    return y

# Implementamos ambos esquemas de enfriamiento
# temp: En ambos esquemas se refiere a la temperatura inicial
# beta y alpha: factores de enfriamiento
# t: numero de iteracion

def decremento_lento(temp,beta=0.1):
    return temp/(1+beta*temp)

def logaritmico(t,temp, alpha=0.95):
    return temp*alpha**t

def enfriamiento(esquema,t,temp,beta,alpha):
    evaluation = None
    if esquema == 'decremento':
        evaluation = decremento_lento(temp,beta)
    elif esquema == 'logaritmico':
        evaluation = logaritmico(t,temp,alpha)
    return evaluation

def vecino(solucion):
    neighbor = np.copy(solucion)
    #Ya que tenemos soluciones binarias y queremos generar un vecino de manera aleatoria
    #basta con generar el cambio de un bit de forma aleatoria
    indice = np.random.randint(len(neighbor))
    neighbor[indice] = 1 - neighbor[indice]
    return neighbor

# Implementamos esquemas de reemplazo
# Generamos una poblacion con la representacion binaria de soluciones
def poblacion(solution,nBit):
    #solution es el tamaño de la solucion que queremos generar
    #nBit es la cantidad de bit que queremos utilizar para representar la poblacion generada
    return np.random.randint(2,size = (solution,nBit))

# Seleccionamos los padres por medio del metodo de la ruleta
def ruleta(values):
    # Nos referimos a las aptitudes que posee cada individuo de la poblacion
    total = np.sum(values)
    probabilities = abs(values/total)
    parents = np.random.choice(np.arange(len(values)),size = 2, p = probabilities)
    return parents

# Operacion de cruza de n puntos
def cruza(parent_1,parent_2):
    cruce = np.random.randint(len(parent_1)-1)
    hijo_1 = np.concatenate((parent_1[:cruce],parent_2[cruce:]))
    hijo_2 = np.concatenate((parent_2[:cruce],parent_1[cruce:]))
    return hijo_1,hijo_2

def mutacion(individuo,probability):
    for i in range(len(individuo)):
        # A cada bit del individuo le asigamos una probabilidad de ser invertido
        # solo muta si esta es mayor a un numero aleatorio generado
        if np.random.rand() < probability:
            individuo[i] = 1 - individuo[i]
    return individuo

def generacional(population,values,new_population,new_values):
    population[:len(new_population)] = new_population
    values[:len(new_population)]= new_values
    return population, values

def elitismo(population,values,new_population,new_values):
    # Con argsort podemos extraer los indices de los valores mas pequeños que tenemos 
    index = np.argsort(values)[:len(new_population)]
    population[index] = new_population
    values[index]= new_values
    return population, values

def peores(population,values,new_population,new_values):
    index = np.flip(np.argsort(values)[:len(new_population)])
    population[index] = new_population
    values[index]= new_values
    return population, values

def reemplazo(esquema,population,values,new_population,new_values):
    remplace = None
    if esquema == 'generacional':
        remplace = generacional(population,values,new_population,new_values)
    elif esquema == 'elitismo':
        remplace = elitismo(population,values,new_population,new_values)
    elif esquema == 'peores':
        remplace = peores(population,values,new_population,new_values)
    return remplace

#Agregamos una funcion auxiliar para evaluar las soluciones
def evalua(function,x):
    evaluation = None
    if function == 'sphere':
        evaluation = sphere(x)
    elif function == 'ackley':
        evaluation = ackley(x)
    elif function == 'griewank':
        evaluation = griewank(x)
    elif function == 'rastrigin':
        evaluation = rastrigin(x)
    elif function == 'rosenbrock':
        evaluation = rosenbrock(x)
    return evaluation

In [166]:
def recocido_simulado(temperatura_inicial,temperatura_final,funcion, dimension,esquema_enfriamiento,beta=0.01,alpha =0.95):
    mejor_solucion = np.random.randint(2,size = dimension)
    mejor_evaluacion = evalua(funcion,mejor_solucion)
    temperatura = temperatura_inicial
    t = 0

    while temperatura > temperatura_final:
        #Generamos el vecino a evaluar
        t += 1
        nueva_solucion = vecino(mejor_solucion)
        nueva_evaluacion = evalua(funcion,nueva_solucion)
        delta = nueva_evaluacion - mejor_evaluacion
        probabilidad = np.exp(-delta/temperatura)
        #Debido a que no todas las funciones necesitan perturbaciones similares
        #establecemos un criterio de aceptacion de probabilidad arbitrario aleatorio
        criterio = random.random()

        if nueva_evaluacion < mejor_evaluacion:
            mejor_solucion = nueva_solucion
            mejor_evaluacion = nueva_evaluacion
            #Si la probabilidad de que la solucion sea mejor es mayor a nuestro criterio
            #aleatorio, entonces aceptamos la nueva solucion generada
        elif probabilidad > criterio:
            mejor_solucion = nueva_solucion
            mejor_evaluacion = nueva_evaluacion

        #Aplicamos el esquema de enfriamiento para reducir la temperatura constantemente
        temperatura = enfriamiento(esquema_enfriamiento,t,temperatura,beta,alpha)
    
    return mejor_solucion, mejor_evaluacion

In [169]:
def algoritmo_genetico(generation,pob, nBit, probability,function,remplace):
    # generation: se refiere al número de iteraciones de realizaremos
    # pob: tamaño de la poblacion que queremos generar
    # nBit: cantidad de Bits con los que queremos representar la poblacion
    # function: funcion que queremos optimizar
    population = poblacion(pob,nBit)
    
    mejor_solucion = []
    mejor_evaluacion = []

    # Aplicamos el criterio de paro, para un numero determinado de generaciones
    for i in range(generation): 
        evaluation = evalua(function,population)
        new_population = []
        new_evaluation = []
        for j in range(pob // 2):
            parents = ruleta(evaluation)
            padre_1,padre_2 =  population[parents]
            hijo_1,hijo_2 = cruza(padre_1, padre_2)
            muta_1 = mutacion(hijo_1, probability)
            muta_2 = mutacion(hijo_2, probability)
            value_1 = evalua(function,muta_1)
            value_2 = evalua(function,muta_2)
            new_evaluation.append(value_1)
            new_evaluation.append(value_2)
            new_population.append(muta_1)
            new_population.append(muta_2)

        new_population = np.array(new_population)
        population, evaluation = reemplazo(remplace,population,evaluation,new_population,new_evaluation)
        index = np.argsort(evaluation)[0]
        mejor_evaluacion.append(min(evaluation))
        mejor_solucion.append(population[index])

    return mejor_evaluacion,mejor_solucion


In [172]:
algoritmo_genetico(30,1000, 10, 0.1,'ackley','elitismo')

([0.0,
  0.0,
  1.2257411716696964,
  1.2257411716696964,
  0.0,
  1.2257411716696964,
  0.0,
  1.2257411716696964,
  1.2257411716696964,
  1.2257411716696964,
  1.2257411716696964,
  0.0,
  1.2257411716696964,
  1.2257411716696964,
  1.7111871278556592,
  0.0,
  1.7111871278556592,
  0.0,
  1.2257411716696964,
  1.2257411716696964,
  1.2257411716696964,
  1.2257411716696964,
  0.0,
  1.2257411716696964,
  1.2257411716696964,
  1.2257411716696964,
  1.2257411716696964,
  0.0,
  1.2257411716696964,
  1.2257411716696964],
 [array([0, 0, 1, 1, 1, 1, 0, 0, 0, 1]),
  array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0]),
  array([0, 1, 1, 1, 0, 0, 1, 1, 0, 0]),
  array([0, 1, 0, 1, 0, 1, 1, 0, 1, 1]),
  array([1, 0, 1, 1, 0, 0, 0, 1, 1, 0]),
  array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]),
  array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1]),
  array([0, 0, 1, 1, 0, 1, 0, 0, 0, 1]),
  array([1, 1, 1, 1, 1, 0, 1, 1, 0, 0]),
  array([0, 1, 1, 0, 0, 1, 0, 1, 0, 0]),
  array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1]),
  array([0, 1, 0, 1, 0,

In [55]:
recocido_simulado(1000,0.2,'griewank', 10,'logaritmico',beta=0.01,alpha =0.95)

(array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1]), 0.6632236008996191)

In [59]:
recocido_simulado(1000,0.2,'griewank', 10,'decremento',beta=0.01,alpha =0.95)

(array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0]), 0.1394508874602628)